In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import random
import numpy as np
import json
import os
import torch.nn.functional as F
from torch.optim import lr_scheduler

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
def get_path(base):
        paths = []
        with os.scandir(base) as entries:
            for entry in entries:
                paths.append(base + '/' + entry.name)
                pass
        return paths

In [4]:
def get_mp(mp_dict, name):
    for mp in mp_dict['points']:
        if mp['name'] == name:
            return mp['center']
    return "mp not found"

In [61]:
############################################## test ######################################################
d = range(5)
ws = 3
step = 2
b = []
temp = []
total_nod = len(d)-ws + 1
for i in range(total_nod):
    for j in range(ws):
        if len(temp) < ws:
            temp.append(d[j+i])
        else:
            b.append(temp)
            temp = []
            temp.append(d[j+i])
b.append(temp)
b

[[0, 1, 2], [1, 2, 3], [2, 3, 4]]

In [42]:
def get_mass_point(joint):
    lshoulder = joint[0]
    rshoulder = joint[1]
    lhip = joint[6]
    rhip = joint[7]
    mid = (lshoulder+rshoulder+lhip+rhip)/4
    for i in range(12):
        joint[i][0] -= mid[0]
        joint[i][1] -= mid[1]
        joint[i] /= 200
    return joint

In [44]:
############################################## test ######################################################
folder_paths = get_path('E:/labeled_data')
json_path = get_path(folder_paths[0])[0]
with open(json_path, 'r') as score_json:
    frame_dict = json.load(score_json)
    
a = np.array(frame_dict['frames'][0]['joint'])
# a = a.astype('int32')
temp = []
for p in range(2):
    temp.append(a[p][5:])
temp = np.array(temp)
for i in range(2):
    temp[i] = get_mass_point(temp[i])
temp.shape

# print(np.array(frame_dict['frames'][0]['joint']).reshape(68,))
# frame_dict['frames'][0]['label']

(2, 12, 2)

In [41]:
def get_mass_point(joint):
    lshoulder = joint[0]
    rshoulder = joint[1]
    lhip = joint[6]
    rhip = joint[7]
    mid = (lshoulder+rshoulder+lhip+rhip)/4
    for i in range(12):
        joint[i][0] -= mid[0]
        joint[i][1] -= mid[1]
        joint[i] /= 200
    return joint

def get_data(root, window):
    data = []
    SOS_token = np.array([3])
    EOS_token = np.array([4])
    SOS_array = -np.ones((1,48))
    EOS_array = np.ones((1,48))
    img_size = [1920, 1080]
    folder_paths = get_path(root)
    for path in folder_paths:
        vid_name = path.split('/')[-1]
        print(vid_name)
        with open('court_midpoint.json', 'r') as mp_json:
            mp_dict = json.load(mp_json)
            center = get_mp(mp_dict, vid_name)
        print(center)
        score_json_paths = get_path(path)
        for json_path in score_json_paths:
            with open(json_path, 'r') as score_json:
                frame_dict = json.load(score_json)
                total_nod = len(frame_dict['frames'])-window + 1
                temp_x = []
                temp_y = []
                for i in range(total_nod):
                    for j in range(window):
                        if len(temp_x) < window:
                            temp = []
                            joint = np.array(frame_dict['frames'][j+i]['joint'])
                            for p in range(2):
                                temp.append(joint[p][5:])        # 不取眼耳鼻
                            temp = np.array(temp)
                            for i in range(2):
                                temp[i] = get_mass_point(temp[i]) # 2, 12, 2
                            temp = np.array(temp).reshape(48,)
#                             temp[0::2] -= center[0]
#                             temp[1::2] -= center[1]
#                             temp[0::2] /= img_size[0]
#                             temp[1::2] /= img_size[0]
#                             temp = temp.tolist()
                            temp_x.append(temp)
                            if frame_dict['frames'][j+i]['label'] == 1 or frame_dict['frames'][j+i]['label'] == 2 or frame_dict['frames'][j+i]['label'] == 0:
                                temp_y.append(frame_dict['frames'][j+i]['label'])
                            else:
                                print(f"{json_path.split('/')[-1]}, {frame_dict['frames'][j+i]['frame']}")
                        else:
                            temp_x = np.array(temp_x)
                            temp_y = np.array(temp_y)
#                             print(temp_x.shape, temp_y.shape)
                            temp_x = np.concatenate((SOS_array, temp_x, EOS_array))
                            temp_y = np.concatenate((SOS_token, temp_y, EOS_token))
#                             print(temp_x.shape, temp_y.shape)
                            data.append((temp_x, temp_y))
                            temp_x = []
                            temp_y = []
                            temp = []
                            joint = np.array(frame_dict['frames'][j+i]['joint'])
                            for p in range(2):
                                temp.append(joint[p][5:])        # 不取眼耳鼻
                            temp = np.array(temp).reshape(48,)
                            temp[0::2] -= center[0]
                            temp[1::2] -= center[1]
                            temp[0::2] /= (img_size[1]/6)
                            temp[1::2] /= (img_size[1]/6)
                            temp_x.append(temp)
                            if frame_dict['frames'][j+i]['label'] == 1 or frame_dict['frames'][j+i]['label'] == 2 or frame_dict['frames'][j+i]['label'] == 0:
                                temp_y.append(frame_dict['frames'][j+i]['label'])
                            else:
                                print(f"{json_path.split('/')[-1]}, {frame_dict['frames'][j+i]['frame']}")
                temp_x = np.array(temp_x)
                temp_y = np.array(temp_y)
                temp_x = np.concatenate((SOS_array, temp_x, EOS_array))
                temp_y = np.concatenate((SOS_token, temp_y, EOS_token))
                data.append((temp_x, temp_y))
    print(len(data))
    return data          

In [118]:
data = get_data('E:/labeled_data', 20)

BWF Thomas Cup  Anthony Sinisuka Ginting (INA) vs Chou Tien Chen (TPE)  Group A
[955, 615]
BWF Thomas Cup  Chou Tien Chen (TPE) vs Kantaphon Wangcharoen (THA)  Group A
[959, 612]
DAIHATSU Indonesia Masters 2021  Chou Tien Chen (TPE) [4] vs Loh Kean Yew (SGP)  Round of 16
[964, 655]
24959


In [119]:
data[99][1]

array([3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [79]:
print(type(data), type(data[0]), type(data[0][0]), type(data[0][1]))
print(data[0][0].shape)
print(data[0][1].shape)

<class 'list'> <class 'tuple'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
(21, 48)
(21,)


In [120]:
def batchify_data(data, batch_size=6, padding=False, padding_token=-1):
    batches = []
    for idx in range(0, len(data), batch_size):
        # We make sure we dont get the last bit if its not batch_size size
        if idx + batch_size < len(data):
            # Here you would need to get the max length of the batch,
            # and normalize the length with the PAD token.
            if padding:
                max_batch_length = 0

                # Get longest sentence in batch
                for seq in data[idx : idx + batch_size]:
                    if len(seq) > max_batch_length:
                        max_batch_length = len(seq)

                # Append X padding tokens until it reaches the max length
                for seq_idx in range(batch_size):
                    remaining_length = max_bath_length - len(data[idx + seq_idx])
                    data[idx + seq_idx] += [padding_token] * remaining_length

            batches.append(data[idx : idx + batch_size])

    print(f"{len(batches)} batches of size {batch_size}")

    return batches

In [121]:
train_dataloader = batchify_data(data)

4159 batches of size 6


In [69]:
print(len(train_dataloader[0]))
print(len(train_dataloader[0][0]))
print(len(train_dataloader[0][0][0]))
print(len(train_dataloader[0][0][1]))
# batch = train_dataloader[0]
for batch in train_dataloader:
    X = []
    y = []
    for i in range(len(batch)):
        X.append(batch[i][0])
        y.append(batch[i][1])
    X = np.array(X)
    y = np.array(y)
    break
    
y_input = y[:,:-1]
y_input.shape

4
2
12
12


(4, 11)

In [122]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5  10,1
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

In [123]:
class Transformer(nn.Module):
    """
    Model from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    # Constructor
    def __init__(
        self,
        num_tokens,
        dim_model,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dropout_p,
    ):
        super().__init__()

        # INFO
        self.model_type = "Transformer"
        self.dim_model = dim_model

        # LAYERS
        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=500
        )
        self.embedding = nn.Embedding(num_tokens, dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
            activation='gelu'
        )
        self.tgt_tok_emb = nn.Embedding(num_tokens, dim_model)
        self.out = nn.Linear(dim_model, num_tokens)
        
    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None):
        # Src size must be (batch_size, src sequence length)
        # Tgt size must be (batch_size, tgt sequence length)

        # Embedding + positional encoding - Out size = (batch_size, sequence length, dim_model)
#         src = self.embedding(src) * math.sqrt(self.dim_model)
        tgt = self.tgt_tok_emb(tgt) * math.sqrt(self.dim_model)
#         tgt[0] = torch.zeros((10,48))
        
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)
        
        # We could use the parameter batch_first=True, but our KDL version doesn't support it yet, so we permute
        # to obtain size (sequence length, batch_size, dim_model),
        src = src.permute(1,0,2)
        tgt = tgt.permute(1,0,2)
        # Transformer blocks - Out size = (sequence length, batch_size, num_tokens)
        transformer_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        out = self.out(transformer_out)
        
        return out
      
    def get_tgt_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask
    
    def create_pad_mask(self, matrix: torch.tensor, pad_token: int) -> torch.tensor:
        # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
        # [False, False, False, True, True, True]
        return (matrix == pad_token)

In [4]:
def batchify_data(data, batch_size=6, padding=False, padding_token=-1):
    batches = []
    for idx in range(0, len(data), batch_size):
        # We make sure we dont get the last bit if its not batch_size size
        if idx + batch_size < len(data):
            # Here you would need to get the max length of the batch,
            # and normalize the length with the PAD token.
            if padding:
                max_batch_length = 0

                # Get longest sentence in batch
                for seq in data[idx : idx + batch_size]:
                    if len(seq) > max_batch_length:
                        max_batch_length = len(seq)

                # Append X padding tokens until it reaches the max length
                for seq_idx in range(batch_size):
                    remaining_length = max_bath_length - len(data[idx + seq_idx])
                    data[idx + seq_idx] += [padding_token] * remaining_length

            batches.append(data[idx : idx + batch_size])

    print(f"{len(batches)} batches of size {batch_size}")

    return batches

def get_mass_point(joint):
    lshoulder = joint[0]
    rshoulder = joint[1]
    lhip = joint[6]
    rhip = joint[7]
    mid = (lshoulder+rshoulder+lhip+rhip)/4
    for i in range(12):
        joint[i][0] -= mid[0]
        joint[i][1] -= mid[1]
        joint[i] /= 200
    return joint

def get_data2(root, window):
    data = []
    SOS_token = np.array([3])
    EOS_token = np.array([4])
    SOS_array = np.ones((1,2,24))
    EOS_array = -np.ones((1,2,24))
    img_size = [1920, 1080]
    folder_paths = get_path(root)
    for path in folder_paths:
        vid_name = path.split('/')[-1]
        print(vid_name)
#         with open('court_midpoint.json', 'r') as mp_json:
#             mp_dict = json.load(mp_json)
#             center = get_mp(mp_dict, vid_name)
#         print(center)
        score_json_paths = get_path(path)
        for json_path in score_json_paths:
            with open(json_path, 'r') as score_json:
#                 print(score_json)
                frame_dict = json.load(score_json)
                total_nod = len(frame_dict['frames'])-window + 1
                temp_x = []
                temp_y = []
                for i in range(total_nod):
                    for j in range(window):
                        if len(temp_x) < window:
                            temp = []
                            joint = np.array(frame_dict['frames'][j+i]['joint'])
                            for p in range(2):
                                temp.append(joint[p][5:])        # 不取眼耳鼻
                            temp = np.array(temp)
                            for i in range(2):
                                temp[i] = get_mass_point(temp[i])
                            temp = temp.reshape(2,24)
#                             temp[0::2] -= center[0]
#                             temp[1::2] -= center[1]
#                             temp[0::2] /= img_size[1]/4
#                             temp[1::2] /= img_size[1]/4
                            temp = temp.tolist()
                            temp_x.append(temp)
                            if frame_dict['frames'][j+i]['label'] == 1 or frame_dict['frames'][j+i]['label'] == 2 or frame_dict['frames'][j+i]['label'] == 0:
                                temp_y.append(frame_dict['frames'][j+i]['label'])
                            else:
                                print(f"{json_path.split('/')[-1]}, {frame_dict['frames'][j+i]['frame']}")
                        else:
                            temp_x = np.array(temp_x)
                            temp_y = np.array(temp_y)
                            temp_x = np.concatenate((SOS_array, temp_x, EOS_array))
                            temp_y = np.concatenate((SOS_token, temp_y, EOS_token))
#                             print(temp_x.shape, temp_y.shape)
                            data.append((temp_x, temp_y))
                            temp_x = []
                            temp_y = []
                            temp = []
                            joint = np.array(frame_dict['frames'][j+i]['joint'])
                            for p in range(2):
                                temp.append(joint[p][5:])        # 不取眼耳鼻
                            temp = np.array(temp)
                            for i in range(2):
                                temp[i] = get_mass_point(temp[i])
                            temp = temp.reshape(2,24)
#                             temp[0::2] -= center[0]
#                             temp[1::2] -= center[1]
#                             temp[0::2] /= img_size[0]
#                             temp[1::2] /= img_size[0]
                            temp_x.append(temp)
                            if frame_dict['frames'][j+i]['label'] == 1 or frame_dict['frames'][j+i]['label'] == 2 or frame_dict['frames'][j+i]['label'] == 0:
                                temp_y.append(frame_dict['frames'][j+i]['label'])
                            else:
                                print(f"{json_path.split('/')[-1]}, {frame_dict['frames'][j+i]['frame']}")
                temp_x = np.array(temp_x)
                temp_y = np.array(temp_y)
                temp_x = np.concatenate((SOS_array, temp_x, EOS_array))
                temp_y = np.concatenate((SOS_token, temp_y, EOS_token))
                
                data.append((temp_x, temp_y))
    print(len(data))
    return data

In [5]:
data = get_data2('E:/labeled_data', 20)
train_dataloader = batchify_data(data)

BWF Thomas Cup  Anthony Sinisuka Ginting (INA) vs Chou Tien Chen (TPE)  Group A
BWF Thomas Cup  Chou Tien Chen (TPE) vs Kantaphon Wangcharoen (THA)  Group A
DAIHATSU Indonesia Masters 2021  Chou Tien Chen (TPE) [4] vs Loh Kean Yew (SGP)  Round of 16
DAIHATSU Indonesia Masters 2021  Kento Momota (JPN) [1] vs Chou Tien Chen (TPE) [4]  SF
F  MS  CHOU Tien Chen (TPE) [1] vs HEO Kwang Hee (KOR)  BWF 2019
41012
6835 batches of size 6


In [36]:
torch.index_select(torch.zeros(6,22,2,48), 2, torch.tensor([0])).shape
torch.cat((torch.zeros(6,22,24),torch.zeros(6,22,24)),2).shape
torch.reshape(torch.zeros(10,6,48),(torch.zeros(10,6,48).shape[0],torch.zeros(10,6,48).shape[1],2,-1)).shape

torch.Size([10, 6, 2, 24])

In [19]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5  10,1
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])
    
# encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
# transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

class Player_encoder_layer(nn.TransformerEncoderLayer):
    def __init__(self, dim_model, nhead):
        super().__init__(dim_model, nhead)
        self.e1 = nn.TransformerEncoderLayer(int(dim_model/2), nhead=nhead)
        self.e2 = nn.TransformerEncoderLayer(int(dim_model/2), nhead=nhead)
        
    def forward(self, src, src_mask, src_key_padding_mask) -> torch.tensor:
#         print(src.shape)
        if len(src.shape) == 4:
            src1 = torch.index_select(src, 2, torch.tensor([0]).to(device)).squeeze(2)
            src2 = torch.index_select(src, 2, torch.tensor([1]).to(device)).squeeze(2)
        elif len(src.shape) == 3:
            src = torch.reshape(src,(src.shape[0],src.shape[1],2,-1))
            src1 = torch.index_select(src, 2, torch.tensor([0]).to(device)).squeeze(2)
            src2 = torch.index_select(src, 2, torch.tensor([1]).to(device)).squeeze(2)
        out1 = self.e1(src1, src_mask, src_key_padding_mask)
        out2 = self.e2(src2, src_mask, src_key_padding_mask)
        out = torch.cat((out1,out2),2)
#         print('o:', out.shape)
        return out
    
class Transformer3(nn.Module):
    """
    Model from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    # Constructor
    def __init__(
        self,
        num_tokens,
        dim_model,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dropout_p,
    ):
        super().__init__()

        # INFO
        self.model_type = "Transformer"
        self.dim_model = dim_model

        # LAYERS
        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=500
        )
        self.embedding = nn.Embedding(num_tokens, dim_model)
        encoder_layer = Player_encoder_layer(dim_model, num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            custom_encoder = self.transformer_encoder,
#             num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
            activation='gelu'
        )
        self.tgt_tok_emb = nn.Embedding(num_tokens, dim_model)
        
    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None):
        # Src size must be (batch_size, src sequence length)
        # Tgt size must be (batch_size, tgt sequence length)
        #   6, 10, 2, 24
        tgt = self.tgt_tok_emb(tgt) * math.sqrt(self.dim_model)
        src = torch.reshape(src,(src.shape[0],src.shape[1],-1))
        #   6, 10, 48
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)
        
        # We could use the parameter batch_first=True, but our KDL version doesn't support it yet, so we permute
        # to obtain size (sequence length, batch_size, dim_model),
        src = src.permute(1,0,2)
        #   10, 6, 48
        tgt = tgt.permute(1,0,2)
#         src = torch.reshape(src,(src.shape[0],src.shape[1],2,-1))
        #   6, 10, 2, 24
        # Transformer blocks - Out size = (sequence length, batch_size, num_tokens)
        out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        
        return out
      
    def get_tgt_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask
    
    def create_pad_mask(self, matrix: torch.tensor, pad_token: int) -> torch.tensor:
        # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
        # [False, False, False, True, True, True]
        return (matrix == pad_token)

In [20]:
def train_loop(model, opt, loss_fn, dataloader):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    model.train()
    total_loss = 0
    n = 0
    for batch in dataloader:
        X_t = []
        y_t = []
        for i in range(len(batch)):
            X_t.append(batch[i][0])
            y_t.append(batch[i][1])
        X_t = np.array(X_t)
        y_t = np.array(y_t)
#         print('x',X_t, 'y', y_t, y_t.shape)
        X, y = torch.tensor(X_t).to(device), torch.tensor(y_t).to(device)
        X = X.to(torch.float32)
        y = y.to(torch.long)

        # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
        y_input = y[:,:-1]   # 4,10
        y_expected = y[:,1:]
        
        # Get mask to mask out the next words
        sequence_length = y_input.size(1)
        tgt_mask = model.get_tgt_mask(sequence_length).to(device)

        # Standard training except we pass in y_input and tgt_mask
        pred = model(X, y_input, tgt_mask)
        
#         next_item = pred.topk(1)[1].view(-1)[-1].item() # num with highest probability
#         next_item = torch.tensor([[next_item]], device=device)
#         print(next_item)
        
        # Permute pred to have batch size first again
        pred = pred.permute(1, 2, 0)    # 4, 5, 10

#         print('Pred: ',pred)
#         print('Expect: ',y_expect)
        
        loss = loss_fn(pred, y_expected)

        opt.zero_grad()
        loss.backward()
        opt.step()
    
        total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

In [21]:
def fit(model, opt, loss_fn, train_dataloader, epochs, scheduler):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    # Used for plotting later on
    train_loss_list = []
    
    print("Training model")
    for epoch in range(epochs):
        print("-"*25, f"Epoch {epoch + 1}","-"*25)
        train_loss = train_loop(model, opt, loss_fn, train_dataloader)
        train_loss_list += [train_loss]
        scheduler.step()
#         validation_loss = validation_loop(model, loss_fn, val_dataloader)
#         validation_loss_list += [validation_loss]
        
        print(f"Training loss: {train_loss:.4f}")
#         print(f"Validation loss: {validation_loss:.4f}")
        print()
        
    return train_loss_list

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Transformer3(
    num_tokens=4, dim_model=48, num_heads=4, num_encoder_layers=6, num_decoder_layers=6, dropout_p=0.1
).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(opt, step_size=5, gamma=0.1)
train_loss_list = fit(model, opt, loss_fn, train_dataloader, 20, scheduler)

Training model
------------------------- Epoch 1 -------------------------
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.Size([22, 6, 48])
o: torch.Size([22, 6, 48])
torch.S

KeyboardInterrupt: 

In [18]:
torch.save(model.state_dict(), 'transformer2.pt')

In [151]:
def predict(model, input_sequence, max_length=20, SOS_token=3, EOS_token=4):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    model.eval()
    
    y_input = torch.tensor([[SOS_token]], dtype=torch.long, device=device)

    num_tokens = len(input_sequence[0])

    for _ in range(max_length):
        # Get source mask
        tgt_mask = model.get_tgt_mask(y_input.size(1)).to(device)
        
        pred = model(input_sequence, y_input, tgt_mask)
#         print('p： ', pred)
        next_item = pred.topk(1)[1].view(-1)[-1].item() # num with highest probability
        next_item = torch.tensor([[next_item]], device=device)

        # Concatenate previous input with predicted best word
        y_input = torch.cat((y_input, next_item), dim=1)

        # Stop if model predicts end of sentence
#         if next_item.view(-1).item() == EOS_token:
#             break

    return y_input.view(-1).tolist()

In [152]:
a = torch.tensor([[[ -0.2695,   0.1639,   0.0984, -21.2954]]])

In [153]:
val_x_list = []
val_y_list = []
for i in range(len(data)):
    val_x_list.append(data[i][0])
    val_y_list.append(data[i][1])    

In [158]:
modele = Transformer2(
    num_tokens=4, dim_model=24, num_heads=2, num_encoder_layers=3, num_decoder_layers=3, dropout_p=0.1
).to(device)
modele.load_state_dict(torch.load('transformer2.pt'))
modele.eval()
for i in range(len(val_x_list)):
    x = torch.tensor(val_x_list[i]).unsqueeze(0).to(device)
    x = x.to(torch.float32)
    result = predict(modele, x)
    print('r',result, len(result))
#     print('v',val_y_list[i], len(val_y_list[i]))

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

...

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 21
r [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2